<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Barcode_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# 2. Baixando a CNN pré-treinada

In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

# 3. Conectando o drive ao colab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# 4. Carregando os arquivos do colab

In [0]:
!unzip "/content/drive/My Drive/Colab/barcode2.zip" 

# 5. Treinando o modelo

In [8]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
trainer.setTrainConfig(object_names_array=["barcode"], batch_size=4, num_experiments=10, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.85
Anchor Boxes generated.
Detection configuration saved in  barcode/json/detection_config.json
Training on: 	['barcode']
Training with Batch Size:  4
Number of Experiments:  10
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Training with transfer learning from pretrained Model


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Instructions for updating:
Use tf.cast instead.
Epoch 1/10
952/952 [==============================] - 978s 1s/step - loss: 23.0772 - yolo_layer_1_loss: 2.4024 - yolo_layer_2_loss: 7.4251 - yolo_layer_3_loss: 13.2497 - val_loss: 8.4703 - val_yolo_layer_1_loss: 1.0759 - val_yolo_layer_2_loss: 3.2490 - val_yolo_layer_3_loss: 4.1453
Epoch 2/10
952/952 [==============================] - 949s 997ms/step - loss: 7.4169 - yolo_layer_1_loss: 1.3553 - yolo_layer_2_loss: 2.7237 - yolo_layer_3_loss: 3.3379 - val_loss: 7.0363 - val_yolo_layer_1_loss: 0.9503 - val_yolo_layer_2_loss: 2.3636 - val_yolo_layer_3_loss: 3.7223
Epoch 3/10
952/952 [==============================] - 927s 974ms/step - loss: 6.0915 - yolo_layer_1_loss: 0.9024 - yolo_layer_2_loss: 2.3319 - yolo_layer_3_loss: 2.8572 - val_loss: 6.9369 - val_yolo_layer_1_loss: 0.8217 - val_yolo_layer_2_loss: 2.1768 - val_yolo_layer_3_loss: 3.9384
Epoch 4/10
952/952 [==============================] - 980s 1s/step - loss: 5.7154 - yolo_layer_1_loss

# 6. Avaliando os modelos gerados

In [9]:
metrics = trainer.evaluateModel(model_path="barcode/models", json_path="barcode/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

Starting Model evaluation....


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Model File:  barcode/models/detection_model-ex-001--loss-0023.077.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.5444
mAP: 0.5444
Model File:  barcode/models/detection_model-ex-002--loss-0007.417.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.8148
mAP: 0.8148
Model File:  barcode/models/detection_model-ex-003--loss-0006.091.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.7886
mAP: 0.7886
Model File:  barcode/models/detection_model-ex-004--loss-0005.715.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.8770
mAP: 0.8770
Model File:  barcode/models/detection_model-ex-005--loss-0005.478.h5 

Using IoU :  0.5
Using Object Threshold :  0.3
Using Non-Maximum Suppression :  0.5
barcode: 0.6657
mAP: 0.6657
Model File:  barcode/models/detection_model-ex-006--loss-0005.068.h5 

Using IoU :  0

# 7. Criando pastas


In [10]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas"))
file_count_corridas = len(files)
print("Corridas: ", file_count_corridas + 1)

file_count_corridas = file_count_corridas+1
newFolderCorrida = file_count_corridas + 1
newFolderCorrida = str(newFolderCorrida)
folder = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida
!mkdir $folder


Corridas:  1


# 8. Setando as variaveis


In [0]:
modelPath = "/content/drive/My Drive/Colab/models/barcode/detection_model-ex-004--loss-0005.715-1.h5"
JsonPath = "drive/My Drive/Colab/json/detection_config_barcode.json"
InputImage = "drive/My Drive/Colab/Imagens_Originais/img1.jpg"

# 9. Treinando modelo e recortando os codigos de barras da imagem original


In [41]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()
detections, extracted_objects_array = detector.detectObjectsFromImage(input_image = InputImage, 
                                                                      output_image_path="barcode-detected.jpg",
                                                                      minimum_percentage_probability=40,
                                                                      extract_detected_objects=True)
def boxPoints(e):
  return e['box_points']

detections.sort(key=boxPoints)

print("Número       " , ": ", "Porcentagem ", "      :    ", "box_points")
print("----------------------------------------------------------------")

print("")

for detection, object_path in zip(detections, extracted_objects_array):
    print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])


print("----------------------------------------------------------------")
print("Quantidade de código de barras detectados na imagem: ", len(detections))

Número        :  Porcentagem        :     box_points
----------------------------------------------------------------

barcode       :  67.15813279151917  :  [181, 1564, 303, 1674]
barcode       :  64.76048827171326  :  [206, 4054, 297, 4158]
barcode       :  67.07512140274048  :  [208, 3368, 313, 3480]
barcode       :  78.62685918807983  :  [231, 3712, 329, 3813]
barcode       :  70.12013792991638  :  [240, 4338, 326, 4465]
barcode       :  64.25415873527527  :  [287, 2349, 394, 2467]
barcode       :  48.5990047454834  :  [299, 2646, 399, 2780]
barcode       :  65.86588025093079  :  [307, 3003, 403, 3108]
barcode       :  40.08749723434448  :  [369, 1936, 476, 2081]
barcode       :  72.16693758964539  :  [392, 1209, 478, 1317]
barcode       :  40.238481760025024  :  [425, 4750, 503, 4854]
barcode       :  64.54731822013855  :  [545, 4329, 627, 4458]
barcode       :  71.56190276145935  :  [576, 3993, 682, 4091]
barcode       :  70.40954828262329  :  [597, 1597, 718, 1690]
barcode      

# 10. Movendo as pastas para o diretório correto.


In [0]:
!mv /content/barcode-detected.jpg $folder
!mv /content/barcode-detected-objects $folder

barcode = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/barcode-detected-objects"
images = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/images_tags"
!mv  $barcode $images